# Leaf Cost and Net Uptake analysis

## Setup

Load up the necessary packages

In [53]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import netcdf
from scipy.signal import find_peaks
plt.rcParams["figure.figsize"]=15,10

Load in the data

In [89]:
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-ckovenparam-cnlf.C137514abc-F92150a45.2020-02-12/run/lnd/hist/concat.nc' # 30 year run
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-ckovenparam-twoll.C137514abc-F99503c9a.2020-02-12/run/lnd/hist/concat.nc' # Only using last two leaf levels
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-leafcost.C137514abc-Fabb2d56e.2020-02-13/run/lnd/hist/concat.nc' # Adding leaf cost to the history output
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-optlai.C137514abc-F5f725c39.2020-02-13/run/lnd/hist/concat.nc' # sla_levleaf calculated from optimium cumulative lai
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-longrun-stndvars.C137514abc-Ffbbcafd0.2020-02-24/run/lnd/hist/concat.nc' # 150 year master branch run
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-tallestoutputonly.C137514abc-F3c05d18c.2020-02-25/run/lnd/hist/concat.nc' #
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-tallestoutputonly-notrimadd.C137514abc-F3c05d18c.2020-02-25/run/lnd/hist/concat.nc' # removed trim inc add for tallest cohort
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-llsf.C137514abc-Fc26723e7.2020-03-03/run/lnd/hist/concat.nc' # 15 year standard run with output to log
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-llsf.C137514abc-Ff596cd86.2020-03-06/run/lnd/hist/concat.nc' # 15 year standard run with output to log and yarly output
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-llsf.C137514abc-Ff596cd86.2020-03-11/run/lnd/hist/concat.nc' # Includes cumulative lai output
# file = '/home/glemieux/scratch/clmed-cases/leaf_flutter-bci-llsf-rev1.C137514abc-Ff596cd86.2020-03-11/run/lnd/hist/concat.nc'
file = '/home/gregorylemieux/Work/Issues/383/concat-031720.nc' # local file

In [3]:
ncfile = netcdf.netcdf_file(file,'r',mmap=False)

### Setup the variables of interest

Copy variables to main memory

In [4]:
# Time
timevar = ncfile.variables['time']
time = timevar[:].copy()
np.shape(time)

(180,)

In [5]:
# TLAI
tlaivar = ncfile.variables['TLAI'] # Total projected leaf area index (for whole patch?)
tlai = tlaivar[:].copy()
np.shape(tlai)

(180, 1)

In [6]:
tlai = np.reshape(tlai,180)

In [7]:
# LAI sun, shade and in canopy by size class
laisun = ncfile.variables['LAISUN_Z_CNLF'][:].copy()
laisha = ncfile.variables['LAISHA_Z_CNLF'][:].copy()
lai_can_scls = ncfile.variables['LAI_CANOPY_SCLS'][:].copy()
np.shape(lai_can_scls)

(180, 36, 1)

In [8]:
laisun = np.reshape(laisun,(180,60))
laisha = np.reshape(laisha,(180,60))
lai_can_scls = np.reshape(lai_can_scls,(180,36))

In [9]:
# Crown area
crownarea_cnlf = ncfile.variables['CROWNAREA_CNLF'][:].copy()
np.shape(crownarea_cnlf)

(180, 60, 1)

In [10]:
crownarea_cnlf = np.reshape(crownarea_cnlf,(180,60))

In [11]:
# Trimming
trim = ncfile.variables['TRIMMING'][:].copy()
trim_can = ncfile.variables['TRIMMING_CANOPY_SCLS'][:].copy()
trim_und = ncfile.variables['TRIMMING_UNDERSTORY_SCLS'][:].copy()
np.shape(trim)
np.shape(trim_can)

(180, 36, 1)

In [12]:
trim = np.reshape(trim,180)
trim_can = np.reshape(trim_can,(180,36))
trim_und = np.reshape(trim_und,(180,36))

In [13]:
# Number of plants
nplants_scls = ncfile.variables['NPLANT_SCLS'][:].copy()
nplants_scls_can = ncfile.variables['NPLANT_CANOPY_SCLS'][:].copy()
nplants_scls_und = ncfile.variables['NPLANT_UNDERSTORY_SCLS'][:].copy()
np.shape(nplants_scls_can)

(180, 36, 1)

In [14]:
nplants_scls = np.reshape(nplants_scls,(180,36))
nplants_scls_can = np.reshape(nplants_scls_can,(180,36))
nplants_scls_und = np.reshape(nplants_scls_und,(180,36))

In [15]:
# Net uptake, timestep
net_uptake_ts = ncfile.variables['NET_C_UPTAKE_CNLF'][:].copy()
# Net uptake, yearly
net_uptake_yrly = ncfile.variables['YRLY_UPTAKE_CNLF'][:].copy()
np.shape(net_uptake_yrly)

(180, 60, 1)

In [16]:
net_uptake_ts = np.reshape(net_uptake_ts,(180,60))
net_uptake_yrly = np.reshape(net_uptake_yrly,(180,60))

In [17]:
# Leaf cost, yearly
leafcost = ncfile.variables['LEAF_COST_CNLF'][:].copy()
np.shape(leafcost)

(180, 60, 1)

In [18]:
leafcost = np.reshape(leafcost,(180,60))

In [19]:
# Cumulative LAI, yearly
cumulative_lai = ncfile.variables['CUMULATIVE_LAI_CNLF'][:].copy()
np.shape(cumulative_lai)

(180, 60, 1)

In [20]:
cumulative_lai = np.reshape(cumulative_lai,(180,60))

Close file

In [21]:
ncfile.close()